<a href="https://colab.research.google.com/github/AleAguirreM/Analitica3AA/blob/main/b_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go
import plotly.express as px
import sys
from mlxtend.preprocessing import TransactionEncoder

In [75]:
#conectamos drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
sys.path.append('/content/drive/MyDrive')

In [77]:
import a_funciones as fn

In [78]:
conn = sql.connect("/content/drive/MyDrive/db_moviesSQL")

In [79]:
curr = conn.cursor()

In [80]:
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

In [81]:
curr.fetchall()

[('ratings',),
 ('movies',),
 ('movies_sel',),
 ('movies_n',),
 ('ratings_final',),
 ('movies_final',)]

In [82]:
pd.read_sql("select * from ratings", conn)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [83]:
#try:
    # Intentar agregar la columna fecha_fecha si no existe
    #curr.execute("ALTER TABLE ratings ADD COLUMN fecha_fecha DATE")
#except sql.OperationalError as e:
    # Si la columna ya existe, mostrar un mensaje de advertencia
    #print("La columna 'fecha_fecha' ya existe en la tabla 'ratings'.")

# Actualizar la nueva columna con los valores convertidos de la columna existente
#curr.execute("UPDATE ratings SET fecha_fecha = DATE(timestamp)")


In [84]:
movies = pd.read_sql("select * from movies", conn)

In [85]:
ratings = pd.read_sql("select * from ratings", conn)

In [86]:

#####Exploración inicial #####

#Identificar campos de cruce y verificar que estén en mismo formato ####
### verificar duplicados

ratings.info()
ratings.head()
ratings.duplicated().sum()

movies.info()
movies.head()
movies.duplicated().sum()

ratings.rating.unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [87]:
#¿Cuántos usuarios han calificado al menos 50 películas diferentes?
consulta_sql = """
    SELECT userId, COUNT(DISTINCT movieId) AS numero_peliculas
    FROM ratings
    GROUP BY userId
    HAVING numero_peliculas >= 50
"""
pd.read_sql(consulta_sql, conn)

,userId,numero_peliculas
0,1,232
1,4,216
2,6,314
3,7,152
4,10,140
...,...,...
380,605,221
381,606,1115
382,607,187
383,608,831


In [88]:
###calcular la distribución de calificaciones
cr=pd.read_sql(""" select
                          "rating" as rating,
                          count(*) as conteo
                          from ratings
                          group by "rating"
                          order by conteo desc""", conn)
cr

,rating,conteo
0,4.0,26818
1,3.0,20047
2,5.0,13211
3,3.5,13136
4,4.5,8551
5,2.0,7551
6,2.5,5550
7,1.0,2811
8,1.5,1791
9,0.5,1370


In [89]:
data  = go.Bar( x=cr.rating,y=cr.conteo, text=cr.conteo, textposition="outside")
Layout=go.Layout(title="Count of ratings",xaxis={'title':'Rating'},yaxis={'title':'Count'})
go.Figure(data,Layout)

Vemos que la pelicula es calificada generalmente cuando es buena o aceptable, la mayor concentracion de calficaciones se encuentra entre 3-5, la distribucion entre califaficaciones de 2.5 a 0.5, es realmente baja.

In [90]:
### calcular cada usuario cuátos peliculas calificó
rating_users=pd.read_sql(''' select "userId" as user_id,
                         count(*) as cnt_rat
                         from ratings
                         group by "userId"
                         order by cnt_rat asc
                         ''',conn )
rating_users

,user_id,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
605,274,1346
606,448,1864
607,474,2108
608,599,2478


In [91]:
fig  = px.histogram(rating_users, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usuario')
fig.show()

En el gràfico vemos que es poco frecuente que los usarios califiquen menos de 500 peliculas, por lo que se acota los datos para ver una mejor distribuciòn. Hay datos atipicos, ya que se muestran usuarios que han calificado más de 2000 peliculas

In [92]:
rating_users.describe()

,user_id,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


En el describe se evidencia que el 75% de los usuarios ha calificado menos de 168 libros, que el minimo de calificaciones es de 20, que la mediana es 165 calificaciones aproximadamente y el total de usuarios es de 610.

In [93]:
### excluimos los usuarios que tengan menos de 170 libros calificados
rating_users2=pd.read_sql(''' select "userId" as user_id,
                         count(*) as cnt_rat
                         from ratings
                         group by "userId"
                         having cnt_rat <=300
                         order by cnt_rat asc
                         ''',conn )

In [94]:
##Graficamos para observar la distribuciòn con la restricciòn
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usario')
fig.show()


In [95]:
rating_users2.describe()

,user_id,cnt_rat
count,527.000000,527.000000
mean,304.197343,85.079696
std,173.397977,68.833582
min,1.000000,20.000000
25%,154.500000,33.000000
50%,306.000000,57.000000
75%,451.500000,118.500000
max,609.000000,300.000000


En este caso vemos una mejor distribucion y la desviacion disminuyò mas de 200 puntos, vemos que hay valores mas cercanas y en ese caso hay 527 datos, es decir, no hubo una reducciòn significativa de datos

In [96]:
#### verificar cuantas calificaciones tiene cada pelicula
rating_movies=pd.read_sql(''' select movieId ,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         order by cnt_rat desc
                         ''',conn )
rating_movies

,movieId,cnt_rat
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
...,...,...
9719,96,1
9720,83,1
9721,77,1
9722,55,1


In [97]:
### analizar distribucion de calificaciones por pelicula
rating_movies.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


En la distribuciòn podemos ver que la pelicula con mas calificaciones es 329 y la mas  baja 1, es decir hay una gran variabiliad, el 75% de las pelicula tiene menos de 9 calificaciones, es decir, generalmente son pocas las calificaciones que recib.

In [98]:
####Filtrar peliculas que tengan mas de 50 calificiones
rating_movies2=pd.read_sql(''' select movieId as movie,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         having cnt_rat>=50
                         order by cnt_rat desc
                         ''',conn )
rating_movies2

,movie,cnt_rat
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
...,...,...
445,3785,50
446,2424,50
447,2105,50
448,910,50


In [99]:
##Anaalizamos la distribucion
rating_movies2.describe()

,movie,cnt_rat
count,450.000000,450.000000
mean,11245.017778,91.911111
std,23457.084170,46.098385
min,1.000000,50.000000
25%,757.000000,59.000000
50%,2020.000000,76.000000
75%,4994.500000,107.750000
max,122904.000000,329.000000


In [100]:
fig  = px.histogram(rating_movies2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada pelicula')
fig.show()


In [101]:
##¿Cuál es la película con el rating promedio más bajo que tiene al menos 100 calificaciones?
avr_min = pd.read_sql("""
    SELECT m.title, AVG(r.rating) AS promedio_calificacion
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.title
    HAVING COUNT(r.rating) >= 100
    ORDER BY promedio_calificacion
    LIMIT 1""",conn)
avr_min

,title,promedio_calificacion
0,Waterworld (1995),2.913043


In [102]:
#### ¿Cuál es la película con la calificación promedio más alta?
consulta_sql = """
    SELECT m.title, AVG(r.rating) as calificacion
    FROM movies m
    LEFT JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.title
    ORDER BY calificacion DESC
    LIMIT 1
"""
pd.read_sql(consulta_sql, conn)

,title,calificacion
0,Zeitgeist: Moving Forward (2011),5.0


In [103]:
#### ¿Cuántos usuarios han calificado al menos 50 películas diferentes?
consulta_sql = """
    SELECT m.title, AVG(r.rating) as calificacion
    FROM movies m
    LEFT JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.title
    ORDER BY calificacion DESC
    LIMIT 1
"""
pd.read_sql(consulta_sql, conn)

,title,calificacion
0,Zeitgeist: Moving Forward (2011),5.0


In [105]:
 fn.ejecutar_sql('/content/drive/MyDrive/prepoce.sql', curr)

In [106]:
curr.execute("select name from sqlite_master where type='table' ")
curr.fetchall()






[('ratings',),
 ('movies',),
 ('movies_sel',),
 ('movies_n',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [107]:
pd.read_sql('select count(*) from movies', conn)
pd.read_sql('select count(*) from ratings', conn)
pd.read_sql('select count(*) from movies_sel', conn)
pd.read_sql('select count(*) from movies_n', conn)
pd.read_sql('select count(*) from ratings_final', conn)
pd.read_sql('select count(*) from movies_final', conn)
pd.read_sql('select count(*) from full_ratings', conn)

,count(*)
0,66945


In [108]:

ratings=pd.read_sql('select * from full_ratings',conn)
print(ratings.duplicated().sum()) # al cruzar tablas a veces se duplican registros
print(ratings.info())
print(ratings.head(10))

0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66945 entries, 0 to 66944
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     66945 non-null  int64  
 1   movieId    66945 non-null  int64  
 2   rating     66945 non-null  float64
 3   timestamp  66945 non-null  int64  
 4   movieId:1  66945 non-null  int64  
 5   title      66945 non-null  object 
 6   genres     66945 non-null  object 
 7   movieId:2  66945 non-null  int64  
 8   cnt_rat    66945 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 4.6+ MB
None
   userId  movieId  rating  timestamp  movieId:1                        title  \
0       1        1     4.0  964982703          1             Toy Story (1995)   
1       1        3     4.0  964981247          3      Grumpier Old Men (1995)   
2       1        6     4.0  964982224          6                  Heat (1995)   
3       1       47     5.0  964983815         47  Seven (a.

In [109]:

movies=pd.read_sql("""select * from movies""", conn)
genres =movies['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)